## <center> Pré-processamento

### Importando Bibliotecas

In [1]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split   
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
from category_encoders import OneHotEncoder
from sklearn.pipeline import Pipeline
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [2]:
import os
os.getcwd()

'c:\\Users\\JPCONCEICAO\\OneDrive - Suzano S A\\Coding\\PROJETO_RECOMENDADOR_CLONAL\\notebooks'

In [4]:
# Importando funções locais
import sys
import os

# Adiciona o caminho da pasta 'scripts' ao sys.path
sys.path.append(os.path.abspath('../pipelines'))

from pre_processor import OutlierRemoverIQR
from model_trainer import train_and_save_model, predict_and_decode
import pre_processor #Funções locais

### Importando dados

Listando as tabelas dentro do .db

In [5]:
from sqlalchemy import create_engine, inspect

# Localizando o banco de dados
caminho_arquivo_db = '..\\data\\pre_processed\\pos_eda.db'

# Criando a engine
engine = create_engine(f'sqlite:///{caminho_arquivo_db}')

# Criando um inspetor
inspetor = inspect(engine)

# Listando os nomes das tabelas
tabelas = inspetor.get_table_names()

# Exibindo os nomes das tabelas
for tabela in tabelas:
    print(tabela)

df_prod


In [6]:
# Carregando as tabelas em DataFrames
df = pd.read_sql('df_prod', con=engine)
# Converter cada nome de coluna para string - Necessário futuramente para as pipelines
df.columns = [str(col) for col in df.columns]

In [7]:
df.drop(columns=['index'], inplace=True)

In [8]:
df.head()

,Região,Material Genético,Altitude,Espaçamento,Id Floresta,Manejo Atual,Zona Climática,avg_vol/ha
0,MA,SUZMA2019,212.0,INDEFINIDO,5.79,REFORMA,M5,116.418876
1,MA,SUZMA2019,208.0,300X330,5.70,CONDUÇÃO,M5,99.761027
2,MA,SUZMA2019,187.0,300X330,5.70,CONDUÇÃO,M5,91.788245
3,MA,SUZMA2019,218.0,300X330,5.02,CONDUÇÃO,M5,121.690072
4,MA,SUZMA2025,232.0,300X330,5.03,CONDUÇÃO,M5,129.941341


Feature Engineering p.II

In [9]:
# Transformando coluna de espaçamento para Nº de Arv/Ha
# Considerações, caso haja erro, considerar a moda (espaçamento mais comum)
pre_processor.calcular_arvores_por_ha(df, 'Espaçamento')

In [10]:
df.head(2)

,Região,Material Genético,Altitude,arv/ha,Id Floresta,Manejo Atual,Zona Climática,avg_vol/ha
0,MA,SUZMA2019,212.0,1111,5.79,REFORMA,M5,116.418876
1,MA,SUZMA2019,208.0,1111,5.70,CONDUÇÃO,M5,99.761027


Avaliando Cardinalidade das colunas com dados object

In [11]:
# Iterando sobre cada coluna do tipo 'object'
for col in df.select_dtypes(include='object').columns:
    print(f"Coluna: {col}")
    print(f'Quantidade de valores únicos nessa coluna: {df[col].nunique()}')
    print(df[col].value_counts())
    print("\n" + "-"*50 + "\n")


Coluna: Região
Quantidade de valores únicos nessa coluna: 3
Região
MA    818
PA    362
TO    261
Name: count, dtype: int64

--------------------------------------------------

Coluna: Material Genético
Quantidade de valores únicos nessa coluna: 60
Material Genético
SUZMA2019    332
SUZA0562     144
AEC0144      136
SUZA0385     117
SUZA1250     111
CO1355        71
BA6021        57
AEC1528       53
SUZBA1922     43
VCC975        37
SUZA1253      35
SUZA0217      33
CO1407        32
SUZA0407      32
SUZA1265      23
AEC0224       18
PESQUISA      17
SUZA1099      16
VES0021       12
SUZBA1083     11
VE41          10
SUZA1135       9
AMC08          8
CO1423         7
SUZA1981       6
SUZA1958       6
SUZMA2025      5
SUZSP1002      5
VCC865         4
IPB2           3
VES0032        3
SUZA1712       3
CO0321         3
GG100          3
EUR            2
SUZMA2037      2
CO1404         2
BA7340         2
GER0680        2
SUZMA2049      2
SUZMA2022      2
IPB1           2
SUZA0540       2
AEC

Redução de cardinalidade

In [12]:
# Aplicando a função para transformar em "outro" todos o valores que ocorrem < 10 vezes.
df = pre_processor.substituir_valores_raros(df, limite=10, excluir_colunas=['Material Genético'])

In [13]:
# Iterando sobre cada coluna do tipo 'object'
for col in df.select_dtypes(include='object').columns:
    print(f"Coluna: {col}")
    print(f'Quantidade de valores únicos nessa coluna: {df[col].nunique()}')
    print(df[col].value_counts())
    print("\n" + "-"*50 + "\n")

Coluna: Região
Quantidade de valores únicos nessa coluna: 3
Região
MA    818
PA    362
TO    261
Name: count, dtype: int64

--------------------------------------------------

Coluna: Material Genético
Quantidade de valores únicos nessa coluna: 60
Material Genético
SUZMA2019    332
SUZA0562     144
AEC0144      136
SUZA0385     117
SUZA1250     111
CO1355        71
BA6021        57
AEC1528       53
SUZBA1922     43
VCC975        37
SUZA1253      35
SUZA0217      33
CO1407        32
SUZA0407      32
SUZA1265      23
AEC0224       18
PESQUISA      17
SUZA1099      16
VES0021       12
SUZBA1083     11
VE41          10
SUZA1135       9
AMC08          8
CO1423         7
SUZA1981       6
SUZA1958       6
SUZMA2025      5
SUZSP1002      5
VCC865         4
IPB2           3
VES0032        3
SUZA1712       3
CO0321         3
GG100          3
EUR            2
SUZMA2037      2
CO1404         2
BA7340         2
GER0680        2
SUZMA2049      2
SUZMA2022      2
IPB1           2
SUZA0540       2
AEC

Aplicando Pipeline para Remover outliers

In [14]:
# Removendo outliers
outlier_remover = OutlierRemoverIQR()
df = outlier_remover.fit_transform(df)

In [15]:
df.head()

,Região,Material Genético,Altitude,arv/ha,Id Floresta,Manejo Atual,Zona Climática,avg_vol/ha
0,MA,SUZMA2019,212.0,1111,5.79,REFORMA,M5,116.418876
1,MA,SUZMA2019,208.0,1111,5.70,CONDUÇÃO,M5,99.761027
2,MA,SUZMA2019,187.0,1111,5.70,CONDUÇÃO,M5,91.788245
3,MA,SUZMA2019,218.0,1111,5.02,CONDUÇÃO,M5,121.690072
4,MA,SUZMA2025,232.0,1111,5.03,CONDUÇÃO,M5,129.941341


#### Avaliação de Multicolinearidade: Identificação e possível remoção de variáveis altamente correlacionadas.

Avaliar esse quesito é importante já que:
- Dificulta a interpretação dos coeficientes;
- Reduz a capacidade preditiva: Overfitting;
- Atrapalha na identificação da influência individual de variáveis, pois 'dizem a mesma coisa' para o modelo;
- Dentre outros problemas.

In [16]:
def filtrar_e_visualizar_correlacao(df, threshold, drop_column=None):
    # Calcula a matriz de correlação
    corr = df.corr()
    
    # Aplica os filtros de limiar, excluindo a correlação perfeita
    filtro = (abs(corr) >= threshold) & (corr != 1.0)
    df_filtrado = corr.where(filtro).dropna(how='all').dropna(axis=1, how='all')
    
    # Remove a coluna e linha especificada, se fornecido
    if drop_column:
        df_filtrado = df_filtrado.drop(index=drop_column, 
                                       errors='ignore').drop(columns=drop_column, 
                                                             errors='ignore')
    
    # Verifica se o DataFrame resultante está vazio antes de plotar
    if not df_filtrado.empty:
        plt.figure()
        sns.heatmap(df_filtrado, annot=True, cmap='coolwarm', center=0)
        plt.show()
    else:
        print("Nenhuma correlação atende ao limiar especificado.")

In [17]:
numeric_features = df.select_dtypes(include=[np.number]).columns.to_list()
filtrar_e_visualizar_correlacao(df[numeric_features], threshold=0.4)

Nenhuma correlação atende ao limiar especificado.


Podemos observar que mesmo as mais altas correlações estão dentro do aceitável.

Multicolinearidade - Fator de Inflação da Variância (VIF)

In [18]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [19]:
# Filtrar apenas as colunas numéricas
df_numeric = df.select_dtypes(include=[np.number])

# Adicionar uma constante (termo de interceptação) ao DataFrame
df_with_constant = add_constant(df_numeric)

# Calcular o VIF para cada variável
vif_data = pd.DataFrame()
vif_data["Variable"] = df_with_constant.columns
vif_data["VIF"] = [variance_inflation_factor(df_with_constant.values, i) for i in range(df_with_constant.shape[1])]

# Exibir os resultados
print(vif_data)

      Variable           VIF
0        const  74741.561987
1     Altitude      1.094923
2       arv/ha      1.012426
3  Id Floresta      1.285364
4   avg_vol/ha      1.191340


Considerações:

VIF ≈ 1: Nenhuma multicolinearidade <br>
1 < VIF ≤ 5: Multicolinearidade moderada. <br>
5 < VIF ≤ 10: Alta multicolinearidade. <br>
VIF > 10: Multicolinearidade muito alta.

Verificamos uma baixa multicolinearidade.

## Treinando modelo

In [24]:
# Exemplo de uso
train_and_save_model(df, target_col='avg_vol/ha', model_path='../models/modelo_treinado.pkl', preprocessor_path='../models/preprocessador.pkl', label_encoder_path='../models/label_encoder.pkl')

Modelo salvo em: ../models/modelo_treinado.pkl
Preprocessador salvo em: ../models/preprocessador.pkl
Label Encoder salvo em: ../models/label_encoder.pkl


In [25]:
df.head(2)

,Região,Material Genético,Altitude,arv/ha,Id Floresta,Manejo Atual,Zona Climática,avg_vol/ha
0,MA,SUZMA2019,212.0,1111,5.79,REFORMA,M5,116.418876
1,MA,SUZMA2019,208.0,1111,5.70,CONDUÇÃO,M5,99.761027


In [33]:
df['Material Genético'].unique()

array(['SUZMA2019', 'SUZMA2025', 'VE41', 'SUZA0562', 'CO1407', 'CO1355',
       'SUZA1265', 'SUZA0385', 'AEC0144', 'PESQUISA', 'SUZA1253',
       'BA6021', 'SUZA1250', 'MA3833', 'SUZA0407', 'SUZA1981', 'AMC08',
       'SUZBA1922', 'AEC0043', 'CO0321', 'EUR', 'SUZA0217', 'SUZA1099',
       'SUZBA1083', 'SUZMA2049', 'SUZA1958', 'BHNPEL011', 'SUZA0540',
       'SUZSP1002', 'SUZA1135', 'SUZA1712', 'SUZA1705', 'AEC1528',
       'BHN0003', 'EPE', 'QGS0149', 'QGS0155', 'AEC0042', 'SUZMA2022',
       'VCC975', 'GG100', 'SUZMA2003'], dtype=object)

#### Métricas de validação

In [23]:
# Treinar e avaliar o modelo
mse, r2, pipeline = train_and_evaluate(X_train, X_test, y_train, y_test)

NameError: name 'train_and_evaluate' is not defined

In [ ]:
# Exibir os resultados
print(f"MSE: {mse}")
print(f"R²: {r2}")

### Importância das features

In [ ]:

importances = model.feature_importances_
feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
print(feature_importances)

           Feature  Importance
4         Altitude    0.327279
0               UP    0.151234
3        Densidade    0.132865
11      (%) Falhas    0.083518
12      avg_vol/ha    0.076364
6      Id Floresta    0.066908
1           Volume    0.052718
10      (%) Mortas    0.032952
8        Município    0.023005
5      Espaçamento    0.019844
9   Zona Climática    0.017708
7     Manejo Atual    0.012662
2           Região    0.002943


**Correlação de Pearson/Spearman**: Para medir a relação entre variáveis contínuas.

**Análise de Variância (ANOVA)**: Para testar a diferença entre grupos categóricos.

**Random Forest**: Utilize a importância das features do Random Forest para identificar os parâmetros mais impactantes.

**Gradient Boosting (XGBoost/LightGBM)**: Outro método que fornece importância das features.

### Modelos de Machine Learning

##### Regressão (Para prever a produtividade)

Random Forest Regressor: Bom para capturar relações não lineares e interações entre variáveis.

Gradient Boosting Regressor (XGBoost/LightGBM): Eficiente para modelos complexos com bom desempenho preditivo.

##### Classificação (Para recomendar espécies)

Random Forest Classifier: Classificador robusto para dados com múltiplas classes.

Gradient Boosting Classifier (XGBoost/LightGBM): Bom para classificações precisas e detalhadas.

### Recomendações e Ajustes

**Validação Cruzada**: Use validação cruzada para garantir que os modelos são robustos.

**Hyperparameter Tuning**: Ajuste os hiperparâmetros dos modelos para melhorar o desempenho.

**Interpretação de Resultados**: Utilize técnicas como SHAP (Shapley Additive Explanations) para interpretar os resultados dos modelos e entender o impacto de cada variável nas previsões.

Criar coluna idade aprox. no momento do corte: (Data entrada-60 dias) - Inicio de plantio 
Criar coluna de TPC(transporte) - data de entrada - Data de corte

Questões

1. Qual material genético produz mais? Há diferenças de acordo com a região?
2. Há interferência do ciclo e rotação na volumetria considerando o material genético?
3. Há relação entre a volumetria e a densidade considerando a mesma área?
4. Que espécie se sai melhor em produtividade em cada região?
5. Qual a variação de volume transportado ao longo do ano?
6. O horário influencia no tempo de carga, permanência no pátio e descarga?
7. A distância influencia muito no TPC?
8. Há alguma correlação entre a densidade e o TPC?
9. Há alguma correlação entre a espécie e o TPC?

Opções: Otimizador ou recomendador

dicas:

Agrupar em macro assuntos

Possibilidades:

Criação de painel de tomada de decisão

Dados extras - Cadastro florestal, puxar o tamanho da UP.